In [34]:
import tensorflow as tf
import tensorflow_hub as hub
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras import layers
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn import preprocessing
from sklearn.model_selection import train_test_split

In [29]:
IMAGE_SHAPE = (224, 224)
TRAINING_DATA_DIR = str(data_root)

In [30]:
datagen_kwargs = dict(rescale=1./255, validation_split=.20)

In [31]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(TRAINING_DATA_DIR,
                                                    subset="validation",
                                                    shuffle=True,
                                                    target_size=IMAGE_SHAPE)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAINING_DATA_DIR,
                                                    subset="training",
                                                    shuffle=True,
                                                    target_size=IMAGE_SHAPE)

Found 111 images belonging to 15 classes.
Found 465 images belonging to 15 classes.


In [32]:
print (train_generator.class_indices)

{'bhuvneshwar_kumar': 0, 'dinesh_karthik': 1, 'hardik_pandya': 2, 'jasprit_bumrah': 3, 'k._l._rahul': 4, 'kedar_jadhav': 5, 'kuldeep_yadav': 6, 'mohammed_shami': 7, 'ms_dhoni': 8, 'ravindra_jadeja': 9, 'rohit_sharma': 10, 'shikhar_dhawan': 11, 'vijay_shankar': 12, 'virat_kohli': 13, 'yuzvendra_chahal': 14}


In [33]:
labels = '\n'.join(sorted(train_generator.class_indices.keys()))
with open('labels.txt', 'w') as f:
    f.write(labels)
!cat labels.txt

bhuvneshwar_kumar
dinesh_karthik
hardik_pandya
jasprit_bumrah
k._l._rahul
kedar_jadhav
kuldeep_yadav
mohammed_shami
ms_dhoni
ravindra_jadeja
rohit_sharma
shikhar_dhawan
vijay_shankar
virat_kohli
yuzvendra_chahal

In [35]:
model = tf.keras.Sequential([hub.KerasLayer("https://tfhub.dev/google/tf2-preview/mobilenet_v2/feature_vector/4",
                                            output_shape=[1280],
                                            trainable=False),
                             tf.keras.layers.Dropout(0.4),
                             tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')])
model.build([None, 224, 224, 3])
model.summary()

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
keras_layer (KerasLayer)     (None, 1280)              2257984   
_________________________________________________________________
dropout (Dropout)            (None, 1280)              0         
_________________________________________________________________
dense_3 (Dense)              (None, 15)                19215     
Total params: 2,277,199
Trainable params: 19,215
Non-trainable params: 2,257,984
_________________________________________________________________


In [36]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer,
              loss='categorical_crossentropy',
              metrics=['acc'])

In [38]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(train_generator, 
                 epochs=100,
                 verbose=1,
                 steps_per_epoch=steps_per_epoch,
                 validation_data=valid_generator,
                 validation_steps=val_steps_per_epoch).history

Epoch 1/100
15/15 [==============================] - 23s 2s/step - loss: 1.0184 - acc: 0.7398 - val_loss: 2.0632 - val_acc: 0.3423
Epoch 2/100
15/15 [==============================] - 23s 2s/step - loss: 0.9935 - acc: 0.7376 - val_loss: 2.0530 - val_acc: 0.3514
Epoch 3/100
15/15 [==============================] - 23s 2s/step - loss: 0.9354 - acc: 0.7505 - val_loss: 2.0420 - val_acc: 0.3514
Epoch 4/100
15/15 [==============================] - 23s 2s/step - loss: 0.8569 - acc: 0.7849 - val_loss: 1.9970 - val_acc: 0.3784
Epoch 5/100
15/15 [==============================] - 23s 2s/step - loss: 0.8273 - acc: 0.7935 - val_loss: 2.0009 - val_acc: 0.3604
Epoch 6/100
15/15 [==============================] - 23s 2s/step - loss: 0.7467 - acc: 0.8409 - val_loss: 1.9861 - val_acc: 0.3604
Epoch 7/100
15/15 [==============================] - 23s 2s/step - loss: 0.7464 - acc: 0.7957 - val_loss: 1.9715 - val_acc: 0.3874
Epoch 8/100
15/15 [==============================] - 23s 2s/step - loss: 0.7059 - a

In [40]:
final_loss, final_accuracy = model.evaluate(valid_generator, steps = val_steps_per_epoch)
print("Final loss: {:.2f}".format(final_loss))
print("Final accuracy: {:.2f}%".format(final_accuracy * 100))

4/4 [==============================] - 5s 1s/step - loss: 2.0809 - acc: 0.4865
Final loss: 2.08
Final accuracy: 48.65%
